In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
import openpyxl
import os
from functions.functions_for_dataload import *
from openpyxl import load_workbook
import seaborn as sns
from scipy import stats
import math

import random 
import time 
import sys 

%matplotlib inline
sys_Data = []
from functions.fnc import *

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
import random 
from keras.models import load_model
from ortools.linear_solver import pywraplp

# Importing Time-Series Data 

In [ ]:
numOfBuses = 14
numOfLines = 20
import pandas as pd
freq = 'sec'
if freq == 'sec':
    senData = pd.read_csv(f"Bus Data\\IEEE_{numOfBuses}_Bus_Time_Series_30_Sec.csv", index_col = 0)
    senData.index = pd.to_datetime(senData.index)
    #senData = senData.iloc[0:25000]
elif freq == 'min':
    senData = pd.read_csv(f"Bus Data\\IEEE_{numOfBuses}_Bus_Time_Series_Hourly.csv", index_col = 0) 
    senData.index = pd.to_datetime(senData.index)
    senData = senData.resample('1T').asfreq().interpolate()
###############################

sensors = numOfBuses + 2* numOfLines

dataset_org = senData.values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset_org)
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
dataset.shape
############
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)
    
look_back = 20
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)
Y_test = scaler.inverse_transform(Y_test)
Y_train = scaler.inverse_transform(Y_train)

print(X_test.shape, Y_test.shape)
################
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[2], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[2], X_test.shape[1]))
############
model = load_model(f'LSTM Model\\IEEE_{numOfBuses}_LSTM.h5')
print("Model Loaded!")
##############
sensorData = scaler.inverse_transform(Y_test)

timeIndx = 0
# LSTM Prediction
def predictLSTM (indexx):
    #numOfZ = 54
    Zpre = pd.DataFrame([])
    Zpre['ID'] = np.arange(1, numOfZ + 1 )
    Zpre['MS'] = scaler.inverse_transform(model.predict(X_test[indexx:indexx+1]))[0]
    Zpre['MS'].iloc[-numOfBuses:] = Zpre['MS'].iloc[-numOfBuses:]*-1
    return Zpre
#predictLSTM (timeIndx)

## Repair_Algorithm(numOfBuses, numOfLines, predicted, received, fixedIDs )

In [ ]:

listOfx = []

def runMatch(numOfBuses, numOfLines, predicted, received, fixedIDs, decoyIDDs):
    
    dataCheck = [predicted.copy(), received.copy()]

    num_sensors = predicted.shape[0]
    num_measures = received.shape[0]

    #costs
    costs = []
    for p in predicted['MS']:
        costs.append(list(abs(received['MS'] - p)))


    for l in fixedIDs:
        try:
            i = pd.Index(predicted['ID']).get_loc(l)
            j = pd.Index(received['ID']).get_loc(l)
            costs[i][j] = 0
        except:
            print(f"Sensor {l} does not exit")
            break 

    th = 10

    # Threshold
    th_max = abs(predicted['MS']*th/100)
    th_max = np.ceil(th_max)
    th_max[th_max == 0] = 1

    numOfSoln = 0
    maxSoln = 1
    
    listOfind = []
#     listOfx = []

    #Stating the loop to find multiple solutions
    while(numOfSoln < maxSoln):
        
        #print("Starting mapping...")

        # Create the mip solver with the CBC backend.
        solver = pywraplp.Solver.CreateSolver('assignment_mip', 'CBC')
        #####################################################################

        # x[i, j] is an array of 0-1 variables, which will be 1
        # if sensor i is assigned to a measurement j.
        x = {}
        for i in range(num_sensors):
            for j in range(num_measures):
                x[i, j] = solver.IntVar(0, 1, '')

        #####################################################################
        # Each sensor is assigned to at most 1 measurement.
        for i in range(num_sensors):
            solver.Add(solver.Sum([x[i, j] for j in range(num_measures)]) <= 1)

        #####################################################################    
        # Each measurement is assigned to at most 1 sensor.
        for j in range(num_measures):
            solver.Add(solver.Sum([x[i, j] for i in range(num_sensors)]) <= 1)

        #####################################################################
        
        # Droping Decoy IDs
        for d in decoyIDDs:
            try:
                j = pd.Index(received['ID']).get_loc(d)
                solver.Add(solver.Sum([x[i, j] for i in range(num_sensors)]) == 0)
            except:
                print(f"Sensor {d} does not exit")
                break        
                
        #####################################################################
        
        # Assigning the fixed IDs
        for l in fixedIDs:
            try:
                i = pd.Index(predicted['ID']).get_loc(l)
                j = pd.Index(received['ID']).get_loc(l)
                solver.Add(x[i,j] == 1)
            except:
                print(f"Sensor {l} does not exit")
                break        
        
#         for i in fixedIDs:
#             solver.Add(x[i-1,i-1] == 1)

        ######################################################################
        # Each received values should be withing the predicted threshold 
        for i in range(num_sensors):
            for j in range(num_measures):
                if costs[i][j]>th_max[i]:
                    solver.Add(x[i,j] == 0)

        # solver.Add(solver.Sum([x[i, j] for i in range(num_workers) 
        #                        for j in range(num_tasks)]) == num_workers )
        # solver.Add(solver.Sum(x) >= 4)

        #######################################################################
        # Finding the next solutions
        if numOfSoln > 0:

            for x_prev in listOfx:
                cond = []
                totalX = sum(sum(x_prev))
                for i in range(num_sensors):
                    for j in range(num_measures):
                        cond.append(x[i,j] * x_prev[i,j])
                solver.Add(solver.Sum(cond) <= totalX-1)                       
        ####################################################################
        # The following code creates the objective function for the problem.

        objective_terms_loss = []
        objective_terms_assign = []

        for i in range(num_sensors):
            for j in range(num_measures):
                objective_terms_loss.append(costs[i][j] * x[i, j])
                objective_terms_assign.append(x[i, j])


        solver.Minimize(solver.Sum(objective_terms_loss) - 2*th*solver.Sum(objective_terms_assign))

        #solver.Minimize(solver.Sum(objective_terms))
        # solver.Maximize(solver.Sum([x[i, j] for i in range(num_workers) for j in range(num_tasks)]))
        #####################################################################

        # The following code invokes the solver.
        status = solver.Solve()

        ######################################################################
        # print the solution
        count = 0
        totCost = 0
        x_values = np.zeros((num_sensors, num_measures), dtype= int)
        recovered_solv = []
        
        recoverData = pd.DataFrame([], columns = ['Mean', 'Var','ID'])
        
        if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
            numOfSoln += 1
            #print('Found a solution!!\nTotal cost = ', solver.Objective().Value(), '\n')
            for i in range(num_sensors):
                found = -1
                for j in range(num_measures):
                    x_values[i,j] = int(x[i,j].solution_value())

                    # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                    if x[i, j].solution_value() > 0.5:
                        found = j
                        count+= 1
    #                     print('Worker %d assigned to task %d.  Cost = %d' %
    #                           (i, j, costs[i][j]))
                        totCost += costs[i][j]

                recovered_solv.append(found+1)
            #print(count, totCost)
            listOfind.append([recovered_solv])
            listOfx.append(x_values)

        
        else:
            print("Cant Solve!!!!!!")
            break

        # Returning result
        #optionsIndx = pd.DataFrame(np.unique(np.array(listOfind)[0]), axis = 0)
        optionsIndx = pd.DataFrame(np.array(listOfind)[0])
        #print(optionsIndx)
        
        for i, c in enumerate (optionsIndx.columns):

            indexx = np.unique(optionsIndx[c])
            indexx = indexx[indexx>0]
            #print(i, c, indexx, indexx.shape)
            if indexx.shape[0] == 0:
                continue
                
            preSensor = predicted['ID'].iloc[c]
            recoverData.loc[preSensor] = [0, 0, 0]
            
            #print("preSensor:", preSensor)
          
#             try:
#                 print("Rec Sensor:")
#                 print(indexx[0]-1)
#                 print(received['ID'].iloc[indexx[0]-1])

#                 #print("Rec Sensor:", optionsIndx[c].iloc[0], received['ID'].iloc[optionsIndx[c].iloc[0]-1])
#             except:
#                 print("Shape:", received['ID'].shape, "But index:", indexx[0]-1)
#                 break
            
            recoverData['ID'].loc[preSensor] = received['ID'].iloc[indexx[0]-1]

            #print('optionsIndx:', optionsIndx[c])
            #print(f"Sensor: {i+1}, Predicted: {predicted['MS'].iloc[i]}, indexx: {indexx}, data: {received['MS'].iloc[indexx-1].values}\n")
            mean = np.mean(received['MS'].iloc[indexx-1].values)
            var = np.square(np.std(received['MS'].iloc[indexx-1].values))
            #print(recoverData)    
            recoverData['Mean'].loc[preSensor] = mean
            #print(recoverData) 
            recoverData['Var'].loc[preSensor] = var
            #recoverData[['MS','Var']].loc[c] = [mean, var]
        recoverData['Var'] = recoverData['Var'] + 1
        recoverData = recoverData.dropna()
        # msrList = np.array(list(recoverData.index))+1
        # recoverData['ID'] = msrList
        ###################
        #print("recoverData: \n", recoverData, '\n\n')
        dataCheck.append(recoverData)

        pre_rec.append(dataCheck)
        ##################
    return recoverData

In [ ]:
def addDecoy(Z_dec, consideredDecoyIDs):
    
    Z_active = Z_dec.copy()
    Z_active[np.array(consideredDecoyIDs), 1] = 0
    
    # State Estimation and Bad Data Detection
    States_decoy, Z_est_decoy, Z_mat_decoy, M_Noise_decoy, Noisy_index_decoy, Rank_decoy, Threshold_decoy = SE_BDD_COR(
        H_org.copy(), Z_active.copy(), W_list, Threshold_min, Threshold_max, Correction, Verbose = "False")

    Z_repeat = Z_dec.copy()
    Z_repeat[consideredDecoyIDs,2] = Z_est_decoy[consideredDecoyIDs].copy()

    for repeat in range(5):
        # State Estimation and Bad Data Detection
        # Running State Estimation
        States_decoy, Z_est_decoy, Z_mat_decoy, M_Noise_decoy, Noisy_index_decoy, Rank_decoy, Threshold_decoy = SE_BDD_COR(
            H_org.copy(), Z_repeat.copy(), W_list, Threshold_min, Threshold_max, Correction, Verbose = "False")
        # Updating decoy Data

        #print(np.linalg.norm(Z_est_decoy[consideredDecoyIDs]-Z_repeat[consideredDecoyIDs,2]))
        Z_repeat[consideredDecoyIDs,2] = Z_est_decoy[consideredDecoyIDs]

        #print(Z_mat_decoy[consideredDecoyIDs,1])
        # Updating decoy Data
        #print(np.linalg.norm(Z_repeat[consideredDecoyIDs,2] - Z_mat[consideredDecoyIDs,2]))
    #print(Z_repeat[consideredDecoyIDs,2])
    return Z_repeat[consideredDecoyIDs,2]

In [ ]:
def filterData(Zrem, Zpre, th, fixedIDDs):
    th = 5
    #print("fixedIDDs", fixedIDDs)
    #print(Zrem.shape, Zpre.shape)
    Zremm = Zrem.copy()
    
    for zr in Zremm.values:
        if zr[0] in fixedIDDs or zr[0] == 0:
            #print("Conitd..", zr[0])
            continue
            
        elif zr[1] == 1:
            pred_value = Zpre['MS'].iloc[int(zr[0]-1)].copy()
            #print("Actual and Predicted", zr[2], pred_value)
            if abs(zr[2] - pred_value) > abs(pred_value*th/100):             
                zr[2] = pred_value.copy()
                #print("Update:", zr[2], pred_value)
    return Zremm   


th = 5
def imputeData(Zrem, Zpre):
    #print(Zrem.shape, Zpre.shape)
    Zremm = Zrem.copy()
    for zr in Zremm.values:
        if zr[1] == -1 or  zr[1] == 0:
            zr[1] = 1
            pred_value = Zpre['MS'].iloc[int(zr[0]-1)].copy()
            #print(zr[2], pred_value)
            zr[2] = pred_value.copy()
    return Zremm  

# Defense Functions

In [ ]:
def defenseEval(Attack_Data, attackIndx, IDbank, attackerLevelList, verbose_ = True):
    

    AttackEval = []
    detectionList = []
    outliers_suspect_List_List= []
    actually_attacked_list = []
    
#     successCount = 0
#     totalAttackedSensors = 0
#     endingIndx = 0
    
#     Sum_list_list = []
#     deviNoiseList_list = []
#     # Starting attacking one by one
#     skipping = 0
    


    attackID = attackIndx #if attackIndx >= 0 else 0
    #print(attackID, numOfAttack)

#     if trial == True and attackID > 10:
#         break
#     if timed == True and skipping<attackID:
#         break
#     if attackID == numOfAttacks:
#         skippingList.append(skipping)

    #################################  

    ###### Randomly Selecting Attacker Level ########
    if attackertype == 3:
        import random
        attackertype_ = random.choice(range(3))
    else: attackertype_ = attackertype

    #--------------------------------------------------------------------------#
    #&&&&&&&&&&&&&&&&&&&&&&&&&& Implementing the attack &&&&&&&&&&&&&&&&&&&&&&&&
    #--------------------------------------------------------------------------#
    attackerLevel = attackerLevelList[attackertype_]

    selectedIDs = IDbank[attackertype_][0]
    deceptiveIDs = IDbank[attackertype_][1]


    #################################################
    if verbose_: #pass 
        print(f"\nattackID: {attackID} \t attackerLevel: {attackertype_}")

    #fixedIDs = Z_mat[1:][np.logical_not(np.array(server[0].deceptive).astype(bool)),0].astype(int)
    #print('attackertype_:', attackertype_)
    
    if attackertype_ == -1:
        selectedIDs = deceptiveIDs.copy()
        Z_dec = Z_mat.copy() 
    
    else:
        # Z_dec is what attacker supposed to see
        Z_dec = Z_mat.copy() 
        Z_dec [selectedIDs, 0] = deceptiveIDs.copy()
        Z_dec = Z_dec [Z_dec[:,0].argsort(kind = 'mergesort')]
    #####################################################
    
    if len(consideredDecoyIDs) > 0 and addDeciy == True:
        Z_dec[consideredDecoyIDs, 2] = addDecoy(Z_dec, consideredDecoyIDs)

    # running state estimation on deceived data ########
    
    # State Estimation and Bad Data Detection
    States_dece, Z_est_dece, Z_mat_dece, M_Noise_dece, Noisy_index_dece, Rank_dece, Threshold_dece = SE_BDD_COR(
        H_org.copy(), Z_dec.copy(), W_list, Threshold_min, Threshold_max, Correction, Verbose = "False")
    #############################################
    
    
    # Preparing the false data to be replaced 
    if attackID >= 0:
        #print("Implementing Attack")
        attackGain = 2
        startingIndx =  (attackID+1) * (numOfZ +1)
        endingIndx = (attackID+2) * (numOfZ +1)
        FData = Attack_Data[startingIndx : endingIndx,:].copy()
    else: 
        #print("No Attack!")
        attackGain = 0
        startingIndx =  (0) * (numOfZ +1)
        endingIndx = (1) * (numOfZ +1)
        FData = Attack_Data[startingIndx : endingIndx,:].copy()
        
    if abs(FData[:,2]).max() > 50:
        FData[:,2] = FData[:,2]*50/abs(FData[:,2]).max()


    injection = FData[:,2]* attackGain
    
    #print(injection, FData[:,2])
    
    FData[:,2] = injection + Z_dec[:,2]


    # Attacker's intend to attack the following sensors
    attackedIndx = np.where(injection != 0)[0]

    # Replacing Z_dec by Z_attack which is received by the EMS 
    Z_att = Z_dec.copy()
    
    if attackCat == 'FDI':
        Z_att[attackedIndx, 2] = FData[attackedIndx, 2].copy()
    elif attackCat == 'DoS':
        #Z_att[attackedIndx, 2] = FData[attackedIndx, 2].copy()
        Z_att[attackedIndx, 1] = -1


    # ------------->>>>>>>>>>>>>>> Filter with the reported sensors only ------------->>>>>>>>>>>>>>>>>>>>>>

    if verbose_: print("Expected to attack: ", attackedIndx)        


    ##--------------------------------------------------------------------------#
    #&&&&&&&&&&&&&&&&&&&&&&&&&& Recovering from the attack &&&&&&&&&&&&&&&&&&&&
    #--------------------------------------------------------------------------#
    if runfilter == True or runImpute == True or recoveryType == 2:
        # Zpre Data from LSTM
        Zpre = predictLSTM (timeIndx)
        #print("timeIndx: ", timeIndx)
        #print(" \n\Zpre:\n", Zpre)
        ##############################
    
    # Received Data after Attack
    received = pd.DataFrame(Z_att[Z_att[:, 1] == 1][1:,[0,2]], columns = ['ID', 'MS'])
    received['ID'] = received['ID'].astype(int)
    received.index = received['ID'] -1
    #print("\n\nreceived:\n", received)

    fixedIDDs = list(set(consideredFixedIDs).intersection(set(received['ID'].values)))
    decoyIDDs = list(set(consideredDecoyIDs).intersection(set(received['ID'].values)))
    #fixedIDDs = consideredFixedIDs
    #print(consideredFixedIDs, received['ID'].values, fixedIDDs)

        
    if recoveryType == 1 or recoveryType == 2:
        #print("Z_att\n\n", Z_att)
        # Derandomizing the received Z_att matrix -- > Z_rec
        Z_rec = Z_att.copy()
        Z_rec[deceptiveIDs, 0] = selectedIDs.copy()
        Z_rec = Z_rec [Z_rec[:,0].argsort(kind = 'mergesort')]
        #print("Z_rec:\n", Z_rec)
        #******************************************************
        Zpair = (Z_mat, Z_rec)
        z_matrices.append(Zpair)
        
    if recoveryType == 2:
        
        Z_back = Z_att.copy()
        Z_back[deceptiveIDs, 0] = selectedIDs.copy()
        Z_back = Z_back [Z_back[:,0].argsort(kind = 'mergesort')]
        
        
        #print(">>>>>>>>>>>>> LSTM <<<<<<<<<<<<")
                
        
        recoverData = runMatch(numOfBuses, numOfLines, Zpre, received, fixedIDDs, decoyIDDs)
        
        ################################################
        Z_rec = Z_att.copy()
        
        ##### find the index of recoered data #####
        recoverdIndx = []
        
        for ids in recoverData['ID']:
            recoverdIndx.append(np.where(Z_rec[:,0] == ids)[0][0])
        
        #print(recoverData['ID'], '\n' ,recoverdIndx)
        
        Z_rec[recoverData.index, 2] = recoverData['Mean']
        Z_rec[1:,1] = -1
        Z_rec[0,1] = 1
        Z_rec[recoverData.index, 1] = 1
        ##########################################
        Z_rec = Z_rec [Z_rec[:,0].argsort(kind = 'mergesort')]
        
        Zpair = (Z_back, Z_rec)
        
        #******************************************************
        
        #print("Difference: ", (Z_rec[[Z_rec[:,1] == 1,2]] - Z_back[[Z_rec[:,1] == 1,2]]).astype(int))

    # Here only the randomization at the attacked layers is shown as the randomization as  
    # rand at the rest of the layers will be recovered without any dataloss, thus skipped
    #print("Z_rec\n", Z_rec)
    
    
    #mapping the attacked locations
    actually_attacked = np.sort((mapOrgID(attackedIndx.copy(), selectedIDs, deceptiveIDs)).astype(int))
    targeted.append(attackedIndx)
    
    if verbose_: print("Actually attacked:", actually_attacked)
    attacked.append(actually_attacked)
    if verbose_: print("Calling State Estimation for recovered data..:")

    # State Estimation and Bad Data Detection
    States_check, Z_est_check, Z_mat_check, M_Noise_check, Noisy_index_check, Rank_check, Threshold_check = SE_BDD_COR(
        H_org.copy(), Z_rec.copy(), W_list, Threshold_min, Threshold_max, Correction, Verbose = "False")
    
    savedState = (States_check, Z_est_check, Z_mat_check, M_Noise_check, Noisy_index_check, Rank_check, Threshold_check)
    
    runningfilter = False; runningImputer = False
    
    Z_rec_df = pd.DataFrame(Z_mat_check, columns = ['ID', 'Taken', 'MS'])
    #########################  Adding LSTM Features ####################
    if Rank_check == True:
        if runfilter == True:
            #print("Filtering Data")
            #print("Z_rec_df:\n", Z_rec_df)
            Z_reco = filterData(Z_rec_df, Zpre, th, fixedIDDs)
            Z_reco = Z_reco.values
            runningfilter = True
            #print("Z_reco:\n", Z_reco)
    else:
        if runImpute == True:
            print("Imputing Data....")
            Z_reco = imputeData(Z_rec_df, Zpre)
            Z_reco = Z_reco.values
            runningImputer = True
            
    if runningfilter == True or runningImputer == True:
        # State Estimation and Bad Data Detection
        States_check, Z_est_check, Z_mat_check, M_Noise_check, Noisy_index_check, Rank_check, Threshold_check = SE_BDD_COR(
            H_org.copy(), Z_reco.copy(), W_list, Threshold_min, Threshold_max, Correction, Verbose = "False")
    
        if runfilter == True and Rank_check == False:
            print("LSTM prediction error!!") if runfilter == True else print("Imputation Failed")
            (States_check, Z_est_check, Z_mat_check, M_Noise_check, Noisy_index_check, Rank_check, Threshold_check) = savedState
            
        if runImpute == True and Rank_check == False:
            print("Imputation failed!!!!!!")
            
            
    #z_est_after_attack.append(Z_est_check.astype(int))
    
    #print("Threshold Threshold_check: ", Threshold_check)
    M_Noise_check = Z_rec[:,2] - Z_est_check
#         print(M_Noise_check[Noisy_index_check])
#         print("\n\n")
    # eliminated attacked sensors
    foundFDI_Idx =  sorted((set(Noisy_index_check)- set(Noisy_index_actu)) & set(actually_attacked))

    # printing noisy indeces
    if Noisy_index_check.size > 0 and verbose_ == True:
        pass
        #print("Noisy indeces before attack: ", Noisy_index_actu)
        #print("Noisy indeces after attack: ", Noisy_index_check)

    # system Unobservable
    if Rank_check == False:
        if verbose_: #pass
            print("-----------  System Unobservable  --------------")
        # Pseudo Deviation 
        #Deviation = -10
        Deviation = 0

        AttackEval.append((attackID+1, "unobservable", Deviation))
#             outliers_suspect_List = [[],[],[]]
#             Sum_list = [[0],[0],[0]]
#             deviNoiseList = [[0],[0],[0]]
        AttackReturn = {}
        #AttackReturn['StatesAttack'] =  States_check
        AttackReturn['StatesAttack'] =  0
        AttackReturn['StatesDeceived'] =  0
        AttackReturn['Deviation']  = 0
        AttackReturn['Check'] = 0
        AttackReturn['Zpair'] = (0,0)
    else:
        # system is observable
        # calculating the percent of deviation in the estiamted measurements
        Deviation = np.linalg.norm(Z_est_check - Z_est_init) #/np.linalg.norm(Z_est_init)*100
        #Deviation = np.linalg.norm(States_check - States_init)/np.linalg.norm(States_init)*100


        ################ detecting/ checking for attack ####################
    
        ####################################################
        attackertype_list.append(attackertype_)

        ########################
        totalAttackedSensors = np.sum(Z_rec[actually_attacked, 1])
        ########################
        successCount = len(set(Noisy_index_check) & set(actually_attacked))
        #########################
        if attackID > 0:
            pass
            #print("Noisy_index_check", Noisy_index_check)
            #print("actually_attacked", actually_attacked)

        # Detected as Bad Data
        if len(foundFDI_Idx) > 0:
            AttackEval = [attackID, "detected", Deviation]
            
            if verbose_: 
                print("!!!!!!!!!! Detected as Bad Data  !!!!!!!!!!!!")
                print("Detected Measurements: ", foundFDI_Idx)

        # Attack was undetected 
        else:
            if verbose_:print("$$$$$$$$$$$$$$  Successfull  $$$$$$$$$$$$$")
            AttackEval = [attackID, "success", Deviation]
        
        #print("skipping: ",skipping)
        # skippingList.append(skipping)
        # converting to matrix

        
        #successCount_avg = successCount / totalAttackedSensors * 100 if totalAttackedSensors > 0 else 0
        AttackReturn = {}
        #AttackReturn['StatesAttack'] =  States_check
        AttackReturn['StatesAttack'] =  Z_est_check
        AttackReturn['StatesDeceived'] =  Z_est_dece
        AttackReturn['Deviation']  = AttackEval[2]
        AttackReturn['Check'] = AttackEval[1]
        AttackReturn['Zpair'] = Zpair

    return AttackReturn

# Initialization

In [ ]:
ASP_list = []
ASEU_list = []
CSER_list = []
EDR_list = []
s_and_d_level = []
s_and_d_sys = []

outlier_list_level = []
numOfBuses_list = [14, 57, 300]
attackertypeList = [-1, 0, 1]
attackerLevelList = ['Level 0', 'Level 1', 'Level 2']


trial = True
localize = False
timed = False
runfilter = False
runImpute = False
addDeciy = False
decoyRepeat = False

#*********** Modeling Attack's capability *************
attacked_Bus = 5
attacked_measurements = 5*attacked_Bus # default

#attackerLevel = attackerLevelList[attackertype]
#######################################################
treeOn = 1
randTypeList = ['random','tree']
randType = randTypeList[treeOn]

#######################################################

# Simulation Repeat
maxRepeat = 1
######################################################

#******************************************************
# Noise threshold for SE-BDD algorithm
Threshold_min = 1
Threshold_max = 1
#######################################################
Correction = False

########################################################
#*********** Modeling Measurement Reading ************
# Modeling Noise --> # mean and sd for Gaussian Noise
noise_mu = 0
noise_sigma_list = [0, 1, 2, 3]

# Percent of sensors reported to the EMS
percentOfreported_list = [50, 60, 70, 80, 90, 100]

# Percent of sensors participated in deception
percentOfDeception_list = [0, 25, 50, 75, 100]
# percentOfFixed_list = [0, 25, 50, 75, 100]

# Starting Program 

In [ ]:
time_df  = pd.DataFrame([])
#######################################################
numOfBuses_list = [57]
attackertypeList = [0,1,2]
#percentOfDeception_list = [75]
#percentOfFixed_list = [0, 25, 50, 75, 100]
percentOfreported_list = [100]
noise_sigma_list = [0]
recoveryType = 1
attackCat = 'FDI'
attackFreq = 5
##################################
init = 500
numdays = 0.01
timestep = int(24*2*60*numdays)
timestep = 250
##################

In [ ]:
numOfAttacks = 250
attackIndxxx  = np.arange(0, numOfAttacks)
np.random.shuffle(attackIndxxx)
# attackIndxxx = np.random.randint(100, size = timestep)
#attackIndxxx[:] = -1

# attackIndxxx = np.arange(1,timestep)
attackIndxxx = attackIndxxx[0:timestep]
print(attackIndxxx)

In [ ]:
pre_rec = []
z_matrices = []
attacked = []
targeted = []
EvalSum = {}

EvalSum['random'] = []
EvalSum['report'] = []
EvalSum['fixed'] = []
EvalSum['attType'] = []
EvalSum['noOfbuses'] = []
EvalSum['noise'] = []
EvalSum['timeStep'] = []
EvalSum['StatesInit'] = []
EvalSum['StatesAttack'] = [] 
EvalSum['StatesDeceived'] = [] 

EvalSum['Deviation'] = [] 
EvalSum['Check'] = [] 
EvalSum['Zpair'] = []
EvalSum['filter'] = []


for q in range(1):
    q = 0
    if q == 0:
        print("Seed-based Remap")
        recoveryType = 1
        runfilter = False
        runImpute = False
        addDecoy = False
    elif q == 1:
        print("Forecast-based Remap")
        recoveryType = 2
        runfilter = False
        runImpute = True
    else:
        print("Hybrid Remap")
        recoveryType = 1
        runfilter = True
        runImpute = True

    for numOfBuses in numOfBuses_list:
        #start_time = time.time()
        print("\nnumOfBuses: ", numOfBuses)

        for attackertype in attackertypeList:
            print("\nattackertype: ", attackertype)

            for noise_sigma in noise_sigma_list:
                print("\nnoise_sigma:", noise_sigma)
                numOfStates = numOfBuses
                ASEU_data = pd.DataFrame([])
                file_name = f"Bus Data\\IEEE_{numOfBuses}.xlsx"

                # Load data into Dataframes
                bus_data_df = pd.read_excel (file_name, sheet_name = "Bus")
                line_data_df = pd.read_excel (file_name, sheet_name = "Branch")

                # number of lines and measurements
                numOfLines = line_data_df.shape[0]
                numOfZ = numOfBuses + numOfLines * 2
                W_list = (numOfZ + 1)*[1]

                # update the index from 1 to number of elements
                bus_data_df.set_index(pd.Series(range(1, numOfBuses+1)), inplace = True)
                line_data_df.set_index(pd.Series(range(1, numOfLines+1)), inplace = True)

                # preprocess data and update line and bus numbers
                #preprocess_data(bus_data_df, line_data_df)

                # print(bus_data_df.head().T)
                # print(line_data_df.head().T)
                ###################################################################
                # Loading Topology Data and Measurement Data
                try:
                    topo_mat = pd.read_excel(file_name, sheet_name = "Topology Matrix")
                    line_data  = pd.read_excel(file_name, sheet_name = "Line Data")
                    print("Topology Matrix Loaded!")
                except:
                    print("Generating Topology Matrix...")
                    topo_mat, line_data = generate_topology_matrix(numOfBuses, numOfLines, line_data_df, file_name)

                Topo = line_data.values.astype(int) #Another name
                ###############################################################

                # Loading Topology Data and Measurement Data
                try:
                    Z_msr_org = pd.read_excel(file_name, sheet_name = "Measurement Data")
                    bus_data = pd.read_excel(file_name, sheet_name = "Bus Data")
                    print("Measurement Data Loaded!")
                except:
                    print("Generating Measurement Data...")
                    Z_msr_org, bus_data = generate_Z_msr_org(numOfBuses, numOfLines, bus_data_df, topo_mat, file_name)



                # Adding IDs and Reported columns
                Z_msr_org.insert(0, 'ID', list(Z_msr_org.index.values))
                Z_msr_org.insert(1, 'Reported', [1]* (numOfZ+1))
                #####################################################

                #************  reading network topo data **************
                fnetname = f'Bus Data//Net_Topo_{numOfBuses}.txt'
                try: netdata = open(fnetname).readlines() 
                except: 
                    print("File Does not exist!")
                ######################################################

                # Load Attack Data, otherwise generate attack data

                file_Name_ = "Attack_Space_"+str(numOfBuses)+"_"+str(numOfLines)+"_"+str(attacked_Bus)+".csv"
                try: 
                    Attack_Data = np.genfromtxt("Attack Data//"+file_Name_, delimiter=',')
                    print("Attack data loaded!")
                except:
                    print("Attack Data is missing! Generating attack data!")
                    current_path = os.getcwd()
                    Attack_Data = generate_attackdata(numOfBuses, numOfLines, line_data, attacked_Bus, current_path)

                attackertype_list = []
                numOfAttacks = int (Attack_Data.shape[0]/(numOfZ+1))
                print("numOfAttacks: ", numOfAttacks)
                meanZ= abs(Z_msr_org['Data']).mean()

                ####################################################################
                if localize == True:
                    #Adding Noisy data
                    Noise_Data = Attack_Data.copy()
                    Noise_Data[:,2] = np.random.randint(-20,20, size = (Noise_Data.shape[0]))
                    Noise_Data[Noise_Data[:,1] == 0, 2] = 0
                    np.random.shuffle(Noise_Data[:,1:])

                    # Updating attack data
                    Attack_Data = np.concatenate((Attack_Data, Noise_Data), axis = 0)


                #******************  sensor cluster mapping ************************************
                sensorIDlist = list(range(1, numOfZ+1, 1))
                clusterIDlist, clusterPopu = assignClusterID (sensorIDlist, Topo, numOfBuses, numOfLines)
                ########################################################################################
                #****************** Starting the loop ***********************

                ###########   List for multiple randomization cases ################
    #                 Attack_Summary_Mean_List = []
    #                 Attack_Summary_Std_List = []
    #                 HistData_List = []
    #                 successCount_Mean =[]
    #                 successCount_Std =[]

    #                 DetectedCases_Mean_List = []
    #                 DetectedCases_Std_List = []

    #                 DetectionAccuracy_Mean_List = []
    #                 DetectionAccuracy_Std_List = []

    #                 successCount_Multi_list = []
    #                 histData_Multi_list = []
    #                 s_and_d = []
    #                 outlier_list_dec = []

                ####################################################################

                for percentOfDeception in percentOfDeception_list:
                    for percentOfreported in percentOfreported_list:

                        print("percentOfDeception: ", percentOfDeception)
                        print("percentOfreported: ", percentOfreported)
                        #print("noise_sigma: ", noise_sigma)


                        ###########  List for Multirun with same randomization #########
                        Attack_Summary_Multi = []
                        histData_Multi = []
                        successCount_Multi =[]

                        DetectedCases_Multi = []
                        DetectionAccuracy_Multi = []
                        ###############################################################

                        # for repeat in range (maxRepeat):
                        repeat = 1
                        retry = 0
                        while(repeat <= maxRepeat):
                            #print("repeat: ",repeat)
                            repeat += 1
                            #------------------------------------------------------------------------------------------------
                            #&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&   Modeling Noise and Report and Deception &&&&&&&&&&&&&&&&&&
                            #------------------------------------------------------------------------------------------------

                            # Percent of observed sensors = 100 - reported 
                            percentOfobserved = 100 - percentOfreported  

                            ##########################    Copying the original Data ###########################
                            H_org = topo_mat.values.copy() 
                            Z_org = Z_msr_org.values.copy()
                            ##################################################################################


                            #############################    Modeling  noise   ###############################
                            Noise = np.random.normal(noise_mu, noise_sigma, numOfZ)
                            Z_org[1:,2] = np.multiply(Z_org[1:,2], Noise/100 + 1)
                            ##################################################################################


                            ############################  Modeling Data Reporting  ###########################
                            randomIDs = np.arange(1,numOfZ)
                            np.random.shuffle(randomIDs)
                            overservedIDs = randomIDs[0: int (percentOfobserved*numOfZ/100)]
                            Z_org[overservedIDs, 1] = 0 # updating Z_org 

                            ### ----------->>  Check observability ---->>>
                            #print(Z_org)
                            Z_mat = Z_org.copy()   
                            ###################################################################################


                            ##########################   Modeling Deception   #################################
                            Deception_flag = []
                            for index in range(Z_mat[1:].shape[0]):
                                Deception_flag.append([])
                            ## Flipping coin each time for each sensor to decide on deception ##########
                            for Z in Z_mat[1:,:]:
                                toss = np.random.binomial(size = 1, n = 1, p = percentOfDeception/100)[0]
                                Deception_flag[Z[0].astype(int) - 1].append(0 if Z[1].astype(int) == 0 else toss)
                            #print(Deception_flag)
                            ####################################################################################


                            #------------------------------------------------------------------------------------------------
                            #&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& SE and BDD &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
                            #------------------------------------------------------------------------------------------------

                            # State Estimation and Bad Data Detection
                            States_init, Z_est_init, Z_mat_init, M_Noise_actu, Noisy_index_actu, fullRank, Threshold = SE_BDD_COR(
                                 H_org.copy(), Z_org.copy(), W_list, Threshold_min, Threshold_max, Correction, Verbose = "False")


                            if fullRank == False:
                                if retry < maxRepeat:
                                    print("The systen is not observable--> Retrying")
                                    repeat -= 1
                                    retry += 1
                                    continue
                                else:
                                    print("The number of reported measurements are too low!")
                                    break

                    #         print("SE threshold for original data : ", Threshold)
                    #         print("Noisy Index in the original data: ", Noisy_index_actu)
                    #         print("fullRank: ", fullRank)
                            ##################################################################################################
                            #&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


                            #------------------------------------------------------------------------------------------------
                            #&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&  Constructing the Network Topology &&&&&&&&&&&&&&&&&&&&&&&&&&& 
                            #-------------------------------------------------------------------------------------------------
                            #start_time = time.time() 
                            ######################### Constrcuting Clusters #################################################
                            lineID = 0
                            clusterList =[]
                            clusterSensorData_List = []
                            shuffleFlagcluster = []

                            while(True):
                                # reading the line
                                line = netdata[lineID]
                                lineID = lineID + 1

                                #checking for # or spaces
                                if line[0] == '#' or line[0] == '\n':
                                    continue

                                # found values
                                else: 
                                    #print(line)
                                    numberOfcluster = int (line.strip())
                                    #print("numberOfcluster:", numberOfcluster)
                                    #adding cluster blank list
                                    for index in range (numberOfcluster):
                                        clusterList.append([])
                                        clusterSensorData_List.append([])
                                        shuffleFlagcluster.append([])
                                    for s, c in zip(sensorIDlist, clusterIDlist):
                                        clusterList[c-1].append(s)
                                    #print("clusterList: ", clusterList)
                                    break

                            ######################### Constrcuting Hubs ########################################
                            hubList = []
                            hubSensorData_List = []
                            shuffleFlagHub = []

                            # calling function to read hub data
                            hubList, hubSensorData_List, shuffleFlagHub, numOfhub, lineID = readX(
                                hubList, hubSensorData_List, shuffleFlagHub, netdata, lineID)
                            #print(hubList)

                            ######################## Constrcuting Servers   ###################################
                            serverList = []
                            serverSensorData_List =[]
                            shuffleFlagServer = []

                            # calling function to read server data
                            serverList, serverSensorData_List, shuffleFlagServer, numOfserver, lineID = readX(
                                serverList, serverSensorData_List, shuffleFlagServer, netdata, lineID)
                            #################################################################################
                            #&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


                            #--------------------------------------------------------------------------------------------
                            #&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&  Setting up Nodes with measurement data &&&&&&&&&&&&&&&&&&&&&
                            #---------------------------------------------------------------------------------------------

                            ##################### initializing sensor nodes ######################################
                            sensor = []
                            for Z in Z_mat[1:]:
                                sensor.append(Node())
                                sensorID = Z[0].astype(int) - 1
                                sensor[sensorID].nodeType = 'Sensor'
                                sensor[sensorID].nodeID = sensorID + 1
                                sensor[sensorID].leaf = True
                                sensor[sensorID].totSensor = 1
                                sensor[sensorID].decSensor = Deception_flag[sensorID][0]
                                sensor[sensorID].remSensor = Deception_flag[sensorID][0]
                                sensor[sensorID].ids.append(sensorID + 1)
                                sensor[sensorID].reported.append(Z[1].astype(int))
                                sensor[sensorID].deceptive.append(Deception_flag[sensorID][0])
                                sensor[sensorID].values.append(Z[2])

                            ##################### initializing cluster nodes ######################################
                            cluster = []
                            for (index, eachcluster) in enumerate(clusterList):
                                #print(eachcluster)
                                cluster.append(Node())

                                cluster[index].nodeType = 'Cluster'
                                cluster[index].nodeID = index + 1
                                for sensorID in eachcluster:
                                    cluster[index].addChild(sensor[sensorID-1])

                            ##################### initializing hub nodes ######################################
                            hub = []
                            for (index, eachhub) in enumerate(hubList):
                                #print(eachhub, )
                                hub.append(Node())
                                hub[index].nodeType = 'Hub'
                                hub[index].nodeID = index + 1
                                for clusterID in eachhub:
                                    hub[index].addChild(cluster[clusterID-1])
                                #hub[index].printNode()

                            ##################### initializing server nodes ######################################
                            server = []
                            for (index, eachserver) in enumerate(serverList):

                                #print(eachserver)
                                server.append(Node())
                                server[index].nodeType = 'Server'
                                server[index].nodeID = index + 1
                                for hubID in eachserver:
                                    server[index].addChild(hub[hubID-1])
                                #server[index].printNode()
                            #######################################################################################


                            #------------------------------------------------------------------------------------------------
                            #&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&  Randmization at each level &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
                            #------------------------------------------------------------------------------------------------

                            # collection of all randomized IDs stored at EMS 
                            IDbank = []

                            # List of org and dec IDs at different level of communication channel
                            orgIDsCluster = []
                            decIDsCluster = []
                            orgIDsHub = []
                            decIDsHub = []
                            orgIDsServer = []
                            decIDsServer = [] 

                            # Randomizing at cluster
                            for clusterID in range(numberOfcluster):
                                orgclusterID, decclusterID = randomizeID(cluster[clusterID], randType)
                                orgIDsCluster += orgclusterID
                                decIDsCluster += decclusterID                

                            # Randomizing at hub
                            for hubID in range(numOfhub):
                                orghubID, dechubID = randomizeID(hub[hubID], randType)
                                orgIDsHub += orghubID
                                decIDsHub += dechubID

                            # Randomizing at server
                            for serverID in range(numOfserver):
                                orgserverID, decserverID = randomizeID(server[serverID], randType)
                                orgIDsServer += orgserverID
                                decIDsServer += decserverID

                            # Merging all the orgIDs and decIDs at diiferent level
                            ID_Stack = [(orgIDsCluster, decIDsCluster),(orgIDsHub, decIDsHub),(orgIDsServer, decIDsServer)]
                            IDbank = func_dec_view(ID_Stack)

                            # Fixed IDs Loop
                            x =pd.DataFrame([])
                            x['ID'] = server[0].ids
                            x['DC'] = server[0].deceptive
                            x['RP'] = server[0].reported
                            filter1 = np.array(x['DC'] == 0)
                            filter2 = np.array(x['RP'] == 1)
                            filterr = np.logical_and(filter1, filter2)
                            fixedIDs = list(x['ID'].loc[filterr])

                            # Fixed IDs Loop

                            
                            
                            initrange = 100-percentOfDeception if decoyRepeat == False else 0
                                                      
                            #initrange = 0
                            
                            for percentOffixed in range(initrange, 100-percentOfDeception+1, 5):
                                                
                                print("percentOffixed: ", percentOffixed)
                                totalFixedPoints = int(numOfZ*percentOffixed/100)

                                consideredFixedIDs = fixedIDs[0:totalFixedPoints]
                                consideredDecoyIDs = fixedIDs[totalFixedPoints:]
                                #print('consideredFixedIDs:',consideredFixedIDs)
                                #print('consideredDecoyIDs:',consideredDecoyIDs)
                            
#                             #initrange = 100-percentOfDeception if recoveryType == 1 else 0
#                             initrange = 100-percentOfDeception
#                             for percentOffixed in range(initrange, 100-percentOfDeception+1, 25):
#                                 print("percentOffixed: ", percentOffixed)
#                                 totalFixedPoints = int(numOfZ*percentOffixed/100)

#                                 consideredFixedIDs = fixedIDs[0:totalFixedPoints]
#                                 #print("FixedIDS: ",fixedIDs, consideredFixedIDs)

                                for timeIndx in range(init, init+timestep):

                                    if timeIndx%attackFreq == 1:
                                        attackIndx = attackIndxxx[timeIndx-init]
                                    else: 
                                        attackIndx = -1

                                    try:
                                        senMsr = Y_test[timeIndx]
                                    except:
                                        senMsr = Z_est_init[1:].copy()
                                    senMsr = np.concatenate((np.array([0]), senMsr), axis =0 )
                                    senMsr[-numOfBuses:] = senMsr[-numOfBuses:]*(-1)                                   
                                    Z_org[:,2] = senMsr
                                    ##################################################################################


                                    #############################    Modeling  noise   ###############################
                                    Noise = np.random.normal(noise_mu, noise_sigma, numOfZ)
                                    Z_org[1:,2] = np.multiply(Z_org[1:,2], Noise/100 + 1)
                                    ##################################################################################
                                    #print(Z_org)
                                    Z_mat = Z_org.copy()   

                                    ###################################################################################
                                    # State Estimation and Bad Data Detection
                                    States_init, Z_est_init, Z_mat_init, M_Noise_actu, Noisy_index_actu, fullRank, Threshold = SE_BDD_COR(
                                         H_org.copy(), Z_org.copy(), W_list, Threshold_min, Threshold_max, Correction, Verbose = "False")


                                    #print("\n\nZ_msr_org:\n", Z_msr_org)


                                    #IDbank = ID_Stack.copy()
                                    #------------------------------------------------------------------------------------------------
                                    #&&&&&&&&&&&&&&&&&&&&&&&&&& Evaluation on State Estimation Attack &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
                                    #------------------------------------------------------------------------------------------------
                                    #time1 = time.time() - start_time
                                    #start_time = time.time() 
                                    ################################  Launching  Attacks ################################
                                    attackEval = defenseEval(Attack_Data, attackIndx, IDbank, attackerLevelList, verbose_ = False)

            #                         AttackEval, Attack_Summary, histDatadeviation, successCount_avg, detectionList, \
            #                         s, d, outlier_sus, outlier_org = defenseEval(
            #                             attacked_Bus, Attack_Data, IDbank, attackerLevelList, attackIndx, verbose_ = False)

                                    #time2 = time.time() - start_time
                                    #time_df [numOfBuses] = [time1, time2]
                                    #print(attackEval)

                                    EvalSum['random'].append(percentOfDeception)
                                    EvalSum['report'].append(percentOfreported)
                                    EvalSum['fixed'].append(percentOffixed)
                                    EvalSum['attType'].append(attackertype)
                                    EvalSum['noOfbuses'].append(numOfBuses)
                                    EvalSum['noise'].append(noise_sigma)
                                    EvalSum['timeStep'].append(timeIndx)
                                    #EvalSum['StatesInit'].append(States_init)
                                    EvalSum['StatesInit'].append(Z_est_init)
                                    EvalSum['StatesDeceived'].append(attackEval['StatesDeceived'])
                                    EvalSum['StatesAttack'].append(attackEval['StatesAttack']) 
                                    EvalSum['Deviation'].append(attackEval['Deviation'])
                                    EvalSum['Check'].append(attackEval['Check'])
                                    EvalSum['Zpair'].append(attackEval['Zpair']) 
                                    EvalSum['filter'].append(q)
                                
EvalSum = pd.DataFrame.from_dict(EvalSum)
fixedList = list(np.unique(EvalSum['fixed']))
randList = list(np.unique(EvalSum['random']))

dev = np.array(EvalSum['StatesInit'] - EvalSum['StatesDeceived'])
dev = np.linalg.norm(np.array([x for x in dev]), axis = 1)
EvalSum['EstDec'] = dev

In [ ]:
EvalSum